# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [4]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [5]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.3 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-3tjz3m5n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-3tjz3m5n
  Resolved https://github.com/huggingface/accelerate.git to commit 1f7a79b428749f45187ec69485f2c966fe21926e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.32.0.dev0-py3-none-any.whl size=311882 sha256=efb2648ce4d314861f3af6205f876680d571dd31eb13f7b5e603275299f75a4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-aejwpamh/wheels/9c/a3/1e/473

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [7]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [8]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True, # i'll make adj here.
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [9]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [11]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [12]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [13]:
sp_nl2sql = sp_nl2sql.format(question="What are the total sales for each book in the last 6 months, and which book has the highest total sales during that time period?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `What are the total sales for each book in the last 6 months, and which book has the highest total sales during that time period?`:
    ```sql3
    


In [14]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [15]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [16]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

WITH book_sales AS (SELECT b.book_id, SUM(s.sale_price) AS total_sales FROM book b JOIN sale s ON b.book_id = s.book_id GROUP BY b.book_id) SELECT book_id, total_sales FROM book_sales WHERE total_sales = (SELECT MAX(total_sales) FROM book_sales) AND total_sales > (SELECT SUM(total_sales) FROM book_sales WHERE book_id IN (SELECT book_id FROM book_sales WHERE total_sales > (SELECT AVG(total_sales) FROM book_sales)));


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [17]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE

    ### Response
    YOUR QERIES AND SAMPLE RESPONSES HERE

    `{question}`:
    ```sql3
    """


In [18]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE

    ### Response
    YOUR QERIES AND SAMPLE RESPONSES HERE

    `Return The name of the best paid employee`:
    ```sql3
    


In [19]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [20]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, MAX(employees.salary) AS max_salary FROM employees GROUP BY employees.first_name, employees.last_name ORDER BY max_salary DESC LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [21]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [22]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return The name of the best paid employee`:
    ```sql3
    


In [23]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [24]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, MAX(employees.salary) AS max_salary FROM employees GROUP BY employees.first_name, employees.last_name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


#Now the question in spanish.


In [25]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Volver El nombre del empleado mejor pagado")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Volver El nombre del empleado mejor pagado`:
    ```sql3
    


In [26]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [27]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT first_name, last_name, SUM(salary) AS total_salary FROM Employee GROUP BY first_name, last_name ORDER BY total_salary DESC LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [28]:
### Instructions:
# Your task is to convert the given natural language question into a SQL query.
# Carefully analyze the provided database schema and use the sample SQL queries to guide your response.

### Database Schema:
products_table = """
CREATE TABLE products (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  category VARCHAR(50),
  price DECIMAL(10,2)
);
"""

sales_table = """
CREATE TABLE sales (
  id INT PRIMARY KEY,
  product_id INT,
  quantity INT,
  sale_date DATE,
  FOREIGN KEY (product_id) REFERENCES products(id)
);
"""

### Sample Queries:
# Get the total sales for each product
total_sales_query = """
SELECT p.name, SUM(s.quantity) AS total_sales
FROM products p
JOIN sales s ON p.id = s.product_id
GROUP BY p.name
ORDER BY total_sales DESC;
"""

# Get the top 3 best-selling products
top_products_query = """
SELECT p.name, SUM(s.quantity) AS total_sales
FROM products p
JOIN sales s ON p.id = s.product_id
GROUP BY p.name
ORDER BY total_sales DESC
LIMIT 3;
"""

### Question:
# What are the total sales for each product category in the last 3 months,
# and which category has the highest total sales during that time period?

### Response:
query = """
SELECT
  p.category,
  SUM(s.quantity * p.price) AS total_sales
FROM sales s
JOIN products p ON s.product_id = p.id
WHERE s.sale_date >= DATE_SUB(CURRENT_DATE, INTERVAL 3 MONTH)
GROUP BY p.category
ORDER BY total_sales DESC
LIMIT 1;
"""
print(query)



SELECT
  p.category,
  SUM(s.quantity * p.price) AS total_sales
FROM sales s
JOIN products p ON s.product_id = p.id
WHERE s.sale_date >= DATE_SUB(CURRENT_DATE, INTERVAL 3 MONTH)
GROUP BY p.category
ORDER BY total_sales DESC
LIMIT 1;



In [29]:
### Instructions:
# Generate a SQL query that answers the provided natural language question.
# Carefully review the database schema and use the sample queries as a guide to construct your response.

### Database Schema:
customers_table = """
CREATE TABLE customers (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  email VARCHAR(255),
  phone VARCHAR(20)
);
"""

orders_table = """
CREATE TABLE orders (
  id INT PRIMARY KEY,
  customer_id INT,
  product_id INT,
  quantity INT,
  order_date DATE,
  FOREIGN KEY (customer_id) REFERENCES customers(id),
  FOREIGN KEY (product_id) REFERENCES products(id)
);
"""

products_table = """
CREATE TABLE products (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  price DECIMAL(10,2)
);
"""

### Sample Queries:
# Get the total order value for each customer
total_order_value_query = """
SELECT c.name, SUM(o.quantity * p.price) AS total_order_value
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN products p ON o.product_id = p.id
GROUP BY c.name
ORDER BY total_order_value DESC;
"""

### Question:
# Which customers have placed orders in the last 30 days,
# and what is the total order value for each of those customers?

### Response:
query = """
SELECT
  c.name,
  SUM(o.quantity * p.price) AS total_order_value
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN products p ON o.product_id = p.id
WHERE o.order_date >= DATE_SUB(CURRENT_DATE, INTERVAL 30 DAY)
GROUP BY c.name
ORDER BY total_order_value DESC;
"""
print(query)



SELECT
  c.name,
  SUM(o.quantity * p.price) AS total_order_value
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN products p ON o.product_id = p.id
WHERE o.order_date >= DATE_SUB(CURRENT_DATE, INTERVAL 30 DAY)
GROUP BY c.name
ORDER BY total_order_value DESC;



In [30]:
### Instructions:
# Convert the provided natural language question into a SQL query that can be executed against the given database schema.
# Use the sample queries as a reference to guide your response.

### Database Schema:
employees_table = """
CREATE TABLE employees (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  department VARCHAR(50),
  salary DECIMAL(10,2)
);
"""

projects_table = """
CREATE TABLE projects (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  start_date DATE,
  end_date DATE
);
"""

employee_projects_table = """
CREATE TABLE employee_projects (
  employee_id INT,
  project_id INT,
  PRIMARY KEY (employee_id, project_id),
  FOREIGN KEY (employee_id) REFERENCES employees(id),
  FOREIGN KEY (project_id) REFERENCES projects(id)
);
"""

### Sample Queries:
# Get the top 3 highest-paid employees
top_employees_query = """
SELECT name, salary
FROM employees
ORDER BY salary DESC
LIMIT 3;
"""

# Get the employees and their assigned projects
employee_projects_query = """
SELECT e.name, p.name AS project_name
FROM employee_projects ep
JOIN employees e ON ep.employee_id = e.id
JOIN projects p ON ep.project_id = p.id;
"""

### Question:
# Find the employees who are currently working on projects that started in the last 3 months,
# and return their names, department, and the project names they are assigned to.

### Response:
query = """
SELECT
  e.name,
  e.department,
  p.name AS project_name
FROM employee_projects ep
JOIN employees e ON ep.employee_id = e.id
JOIN projects p ON ep.project_id = p.id
WHERE p.start_date >= DATE_SUB(CURRENT_DATE, INTERVAL 3 MONTH)
"""
print(query)



SELECT
  e.name,
  e.department,
  p.name AS project_name
FROM employee_projects ep
JOIN employees e ON ep.employee_id = e.id
JOIN projects p ON ep.project_id = p.id
WHERE p.start_date >= DATE_SUB(CURRENT_DATE, INTERVAL 3 MONTH)



**Summary:**
 It's good approach to leveraging large language models for generating SQL queries from natural language prompts. By fine-tuning the Mistral model on SQL generation, the model has been optimized to excel at this task. The code includes detailed steps for loading and configuring the model.


 The key aspect of this exercise is really all about exploring different ways of phrasing the prompts you give to the model. This is such a crucial part of getting the most out of these large language models, i also got to try prompts with different levels of detail.

By exploration of prompt variations, get to a comprehensive understanding of what the model is capable of. Carefully analyzing the outputs it generates is so important.



**practical lesson:**

The impact of INT8/INT4 Quantization for fine tuning.

Accelerate library.

Write a query in diffrent way.